> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.0`: sampling mode demo
The *reinforcement learning* mode can be used to train an agent to find molecules that maximize a user-defined score (typically comprised by a number of score components - check out the respective notebooks). While each iteration of the agent will output a number of molecules (batch), it might be necessary to generate more afterwards, e.g. if more ideas are required to push a project forward. This can be achieved with the *sampling* running mode of `REINVENT 3.0`, which takes a (trained) agent and generates more ideas which is illustrated by this notebook.

To proceed, please update the following code block such that it reflects your system's installation and execute it.

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/reinventcli")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent.v3.0")
output_dir = os.path.expanduser("~/Desktop/REINVENT_sampling_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## Setting up the configuration
`REINVENT` has an entry point that loads a specified `JSON` file on startup. `JSON` is a low-level data format that allows to specify a fairly large number of parameters in a cascading fashion very quickly. The parameters are structured into *blocks* which can in turn contain blocks or simple values, such as *True* or *False*, strings and numbers. In this tutorial, we will go through the different blocks step-by-step, explaining their purpose and potential values for given parameters. Note, that while we will write out the configuration as a `JSON` file in the end, in `python` we handle the same information as a simple `dict`.

In [2]:
# initialize the dictionary
configuration = {
    "version": 3,                          # we are going to use REINVENT's newest release
    "run_type": "sampling"                 # other run types: "scoring", "validation",
                                           #                  "transfer_learning",
                                           #                  "reinforcement_learning" and
                                           #                  "create_model"
}

In [3]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://127.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_path": os.path.join(output_dir, "progress.log"), # where the run's output is stored
    "job_name": "Scoring mode demo",       # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to "remote"
}

We will need to specify a path to an agent (parameter `model_path`), which can be a prior or trained agent. For the purpose of this notebook, we will use a prior shipped with the `REINVENT 3.0` repository.

In [4]:
# set up the file path, where the generated molecules are
# deposited (as SMILES)
output_SMILES_path = os.path.join(output_dir, "sampled", "sampled.smi")

# add the "parameters" block
configuration["parameters"] = {
    "model_path": os.path.join(ipynb_path,        # path to prior or trained agent
                               "models",
                               "random.prior.new"),
    "output_smiles_path": output_SMILES_path,       # output path where SMILES will be deposited
    "num_smiles": 1024,                             # specifies, how many molecules are to be sampled
    "batch_size": 128,                              # internal batch size; should match the one used
                                                    # during training
    "with_likelihood": False                        # also provide the log-likelihood
}

We do not need to add any scoring function definition (in contrast to the e.g. *reinforcement learning* or *scoring* running modes). The reason is that the agent has (hopefully) "learned" to find resonable molecules (defined by the scoring function) and stores that knowledge internally in its weights.

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [5]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "sampling_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## Run `REINVENT`
Now it is time to execute `REINVENT` locally. As we will not update any weights, execution should be very fast. The result will be a `SMILES` file in the `sampled` directory specified. If you want to generate the scores for these, you can make use of the *scoring mode* (see respective notebook for more details).

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent.v3.0

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [6]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path}

In [7]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

In [8]:
# print the resulting SMILES file
!head -n 15 {output_dir}/sampled/sampled.smi

c1cccc(-c2c3c(nc(-c4ccccc4)c2)ccc(Br)c3)c1
C(=O)(c1cc(C(C)C)c(O)cc1)c1cc(CCC(=O)N2CCNCC2)c(O)cc1
c1(C(Nc2cc(C(Nc3ccc(N(C)C)cc3)=O)ccc2)=O)ccccc1
c1n[nH]c2sccc12
COc1c(OC)c(C(OC(CN2CCOCC2)C)=O)ccc1
o1nc(C)c(-c2ccc(-c3noc(C4CCCCN4C(=O)COc4ccccc4)n3)cn2)c1C
[nH]1c(=O)c2ncn(COCC)c2nc1
C1CN(C(C(N=c2[nH]c3c(s2)CCCC3)=O)C)CCN1C(Cc1cncnc1)=O
c1(Nc2[nH]c(=N)c(C#N)c(Cl)c2Cl)c(F)cc(-n2cnc(C)c2)c(OC)c1
c1(-c2c(Cl)cccc2)nc(Cl)c(SC)c(=O)[nH]1
O=[N+]([O-])c1c(S(=O)(=O)N(c2ccccc2)CC(=O)NN=Cc2ccc(SC)cc2)cccc1
Clc1cc2c(cc1)[nH]c(-c1ccc(C=CC(=O)O)cc1)n2
O(c1c(C)cc(C)cc1C)C(C)C1C2(C)C(CC1)C1Cn3c4c(nc3C3C1=C(CCC3)CC(=O)CC4)C2=O
c1c(O)cc(Cl)c(C2=Nc3ccc(F)cc3SC2C)c1
c1c(CC(NS(c2cc(-c3nonc3C)ccc2C)(=O)=O)C(O)=O)cccc1
